# Another implementation of word2vec: this is the first version that I saw

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import collections
import math
import numpy as np
import os
import random
import tensorflow as tf
import inspect
import string
import time
from word2vec import Config, DataHolder

In [27]:
filename = 'pt96.txt'  
file_path = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
file_path = os.path.join(file_path,'data')
file_path = os.path.join(file_path,filename)

my_data = DataHolder(file_path)
vocab_size = 50000

In [28]:
data, count, word2index, index_to_word = my_data.build_data(vocab_size)

In [29]:
for i in data[:100]:
    print(index_to_word[i], end = " ")

Conto Contos UNK 1870 Contos UNK Textofonte Obra Completa Machado de Assis vol II Rio de Janeiro Nova Aguilar 1994 Publicado originalmente pela Editora Garnier Rio de Janeiro em 1870 ÍNDICE UNK UNK UNK UNK A UNK DE UNK O UNK DE UNK UNK DE UMA UNK UNK UNK UNK E UNK UNK FREI UNK UNK UNK ÍNDICE Capítulo Primeiro Capítulo II Capítulo iii Capítulo UNK Capítulo v Capítulo UNK Capítulo UNK CAPÍTULO VIII CAPÍTULO PRIMEIRO Era conveniente ao romance que o leitor ficasse muito tempo sem saber quem era Miss Dollar Mas por outro lado sem a apresentação de 

In [ ]:
data_index = 0

def generate_batch(batch_size, num_skips, skip_window,data):
  global data_index
  assert batch_size % num_skips == 0
  assert num_skips <= 2 * skip_window
  batch = np.ndarray(shape=(batch_size), dtype=np.int32)
  labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
  span = 2 * skip_window + 1 # [ skip_window target skip_window ]
  buffer = collections.deque(maxlen=span)
  for _ in range(span):
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  for i in range(int(batch_size / num_skips)):
    target = skip_window  # target label at the center of the buffer
    targets_to_avoid = [ skip_window ]
    for j in range(num_skips):
      while target in targets_to_avoid:
        target = random.randint(0, span - 1)
      targets_to_avoid.append(target)
      batch[i * num_skips + j] = buffer[skip_window]
      labels[i * num_skips + j, 0] = buffer[target]
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  return batch, labels

batch, labels = generate_batch(batch_size=8, num_skips=2, skip_window=1,data=data)
for i in range(8):
  print(batch[i], '->', labels[i, 0])
  print(index_to_word[batch[i]], '->', index_to_word[labels[i, 0]])

In [35]:
batch_size = 128
embedding_size = 128 # Dimension of the embedding vector.
skip_window = 1 # How many words to consider left and right.
num_skips = 2 # How many times to reuse an input to generate a label.
# We pick a random validation set to sample nearest neighbors. here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent. 
valid_size = 16 # Random set of words to evaluate similarity on.
valid_window = 100 # Only pick dev samples in the head of the distribution.
valid_examples = np.array(random.sample(range(valid_window), valid_size))
num_sampled = 64 # Number of negative examples to sample.

graph = tf.Graph()
with graph.as_default():
    # Input data.
    train_dataset = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
  
    # Variables.
    embeddings = tf.Variable(
    tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0))
    softmax_weights = tf.Variable(tf.truncated_normal([vocab_size, embedding_size],
                         stddev=1.0 / math.sqrt(embedding_size)))
    softmax_biases = tf.Variable(tf.zeros([vocab_size]))
  
    # Model.
    # Look up embeddings for inputs.
    embed = tf.nn.embedding_lookup(embeddings, train_dataset)
    # Compute the softmax loss, using a sample of the negative labels each time.
    loss = tf.reduce_mean(tf.nn.sampled_softmax_loss(softmax_weights, softmax_biases,train_labels, embed,num_sampled, vocab_size))

    # Optimizer.
    optimizer = tf.train.AdagradOptimizer(1.0).minimize(loss)
  
    # Compute the similarity between minibatch examples and all embeddings.
    # We use the cosine distance:
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm
    valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
    similarity = tf.matmul(valid_embeddings, tf.transpose(normalized_embeddings))

In [37]:
num_steps = 100001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  ts = time.time()
  print("Initialized")
  average_loss = 0
  for step in range(num_steps):
    batch_data, batch_labels = generate_batch(
      batch_size, num_skips, skip_window,data)
    feed_dict = {train_dataset : batch_data, train_labels : batch_labels}
    _, l = session.run([optimizer, loss], feed_dict=feed_dict)
    average_loss += l
    if step % 2000 == 0:
      if step > 0:
        average_loss = average_loss / 2000
      # The average loss is an estimate of the loss over the last 2000 batches.
      print("Average loss at step", step, ":", average_loss)
      average_loss = 0
    # note that this is expensive (~20% slowdown if computed every 500 steps)
    if step % 10000 == 0:
      sim = similarity.eval()
      for i in range(valid_size):
        valid_word = index_to_word[valid_examples[i]]
        top_k = 8 # number of nearest neighbors
        nearest = (-sim[i, :]).argsort()[1:top_k+1]
        log = "Nearest to %s:" % valid_word
        for k in range(top_k):
          close_word = index_to_word[nearest[k]]
          log = "%s %s," % (log, close_word)
        print(log)
  te = time.time()
  final_embeddings = normalized_embeddings.eval()

Initialized
Average loss at step 0 : 7.46859169006
Nearest to nas: Terry, Magma, Reinmuth, Dulce, Davenport, Boulevard, SI, rajada,
Nearest to São: emprestando, Frozen, Juntos, atualizado, alicerces, Supostamente, exoplaneta, Padroeiro,
Nearest to ela: Pasadena, Camélias, acusação, B, metafísico, penetrando, atrapalhou, Almada,
Nearest to ele: Mileena, cursar, Kobayashi, branca’, Itálico, la, frei, género,
Nearest to sobre: L, Mendel, Metropolia, momentaneamente, prodígios, Cargo, concentrarse, paróquias,
Nearest to As: Evarista, destinatários, Law, instalação, Papai, queixas, dublagem, ajudados,
Nearest to que: Formalmente, comuna, 1959, Jules, Viajou, Uniformes, respeita, islandês,
Nearest to onde: estelar, Josephson, mortes, materno, 283, Peças, implantados, matrículas,
Nearest to todos: diferenciais, pressionando, exaltação, assiduidade, decretolei, média, válvulas, Borg,
Nearest to em: shiitake, Construído, pulmões, 1434, Business, opus, Gestapo, daquilo,
Nearest to ano: Arafat, a

In [38]:
print("duration= ", te-ts)

duration=  209.96710753440857


In [39]:
from scipy import spatial

def analogy(w1,w2,w3,index_to_word,word2index, embeddings):
    a = embeddings[word2index[w1]]
    b = embeddings[word2index[w2]]
    c = embeddings[word2index[w3]]
    def apply_dot(x):
        return b.dot(x) - a.dot(x) + c.dot(x)
    all_results = [(apply_dot(w), index) for index, w in enumerate(embeddings) if (index!= word2index[w1] and index!= word2index[w2] and index!= word2index[w3])]
    all_results.sort(reverse=True)
    result = [(index_to_word[index],value) for (value,index) in all_results[0:11]]
    return result[0][0]  

def top_k_sim(vector, index_to_word, embeddings,k=10):
    all_sim = [(1 - spatial.distance.cosine(vector, w),index) for index, w in enumerate(embeddings)]
    all_sim.sort(reverse =True)
    result = [(index_to_word[index],value) for (value,index) in all_sim[0:k+1]]
    return result 

In [41]:
mulher = final_embeddings[word2index['mulher']]
top_k_sim(mulher,index_to_word,final_embeddings)

[('mulher', 0.99999994039535522),
 ('moça', 0.43050500750541687),
 ('sepultura', 0.42927879938139923),
 ('mãe', 0.40767711400985718),
 ('irradiação', 0.39029717445373535),
 ('filha', 0.3781704306602478),
 ('tocara', 0.37797605991363525),
 ('pessoa', 0.37743139266967773),
 ('carta', 0.35732680559158325),
 ('senhora', 0.35483704747712053),
 ('derramar', 0.35342999059796443)]

In [46]:
eval_file="AnalogiesBr_little.txt"
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
prefix = os.path.join(currentdir, "evaluation")
file_path = os.path.join(prefix, eval_file)
valid_tests = 0
correct_answer = 0
total_lines = 0
total_loss = 0


In [47]:
import time
import sys
initial_time = time.time()
with open(file_path) as inputfile:
    for line in inputfile:
        total_lines += 1
        initial = time.time()
        list_line = line.strip().split()
        if all([word in word2index for word in list_line]):
            valid_tests += 1
            analogue = analogy(list_line[0],
                               list_line[1],
                               list_line[2],
                               index_to_word,
                               word2index,
                               final_embeddings)
            print("\nAnalogy -->", list_line)
            print("prediction -->", analogue)
            if analogue == list_line[3]:
                correct_answer += 1
                print("\nYESSSSSSSSSSSSSSSSSSSSS\n")
            current_time = time.time() - initial
            sys.stdout.write('\rcurrent_line:{}, duration = {}'.format(total_lines,
                                                                        current_time))
            sys.stdout.flush()

print("\ntotal_lines = {}".format(total_lines))
print("valid_tests = {}".format(valid_tests))
print("correct_answer = {}".format(correct_answer))
print("total_loss = {}".format(total_loss))
print("duration = {}".format(time.time() - initial_time))



Analogy --> ['rapaz', 'moça', 'irmão', 'irmã']
prediction --> mãe
current_line:2, duration = 0.17698097229003906
Analogy --> ['rapaz', 'moça', 'irmãos', 'irmãs']
prediction --> mãe
current_line:3, duration = 0.1764235496520996
Analogy --> ['rapaz', 'moça', 'pai', 'mãe']
prediction --> mãe

YESSSSSSSSSSSSSSSSSSSSS

current_line:4, duration = 0.17835116386413574
Analogy --> ['rapaz', 'moça', 'pai', 'mãe']
prediction --> mãe

YESSSSSSSSSSSSSSSSSSSSS

current_line:5, duration = 0.17598581314086914
Analogy --> ['rapaz', 'moça', 'avô', 'avó']
prediction --> mãe
current_line:6, duration = 0.17580056190490723
Analogy --> ['rapaz', 'moça', 'avô', 'avó']
prediction --> mãe
current_line:7, duration = 0.17597723007202148
Analogy --> ['rapaz', 'moça', 'neto', 'neta']
prediction --> Kaden
current_line:8, duration = 0.17362046241760254
Analogy --> ['rapaz', 'moça', 'noivo', 'noiva']
prediction --> mãe
current_line:9, duration = 0.1723792552947998
Analogy --> ['rapaz', 'moça', 'ele', 'ela']
predictio

26/342